# Serverless Example
## PyTorch [CLASSIFICATION]

## Setup

In [0]:
# Install some dependencies
!pip install torch


import time
import logging
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Use CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__

'1.1.0'

### Load Database

In [0]:
iris = datasets.load_iris()

X = iris.data.astype('float32')
y = iris.target.astype('int')

## Training Process

### DataLoaders definition

In [0]:
BATCH_SIZE = 8

data_loader = DataLoader(dataset=TensorDataset(torch.from_numpy(X), torch.from_numpy(y)),
                         batch_size=BATCH_SIZE, 
                         shuffle=True)

### Model Definition

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=4, out_features=5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

### Training
The model will give bad performances due to the lack of normalization of the inputs.

In [0]:
EPOCHS = 10
MODEL_PATH = 'model'

# Time to train
for e in range(EPOCHS):
    model.train()
    for i, (data, label) in enumerate(data_loader):
        data = data.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, label)
        loss.backward()
        optimizer.step()
        
        print('Train Epoch: {:3d} [{:3d}/{:3d} ({:.0f}%)]\tLoss: {:.6f}'.format(e, 
                                                                       i * len(data), 
                                                                       len(data_loader.dataset),
                                                                       100. * i / len(data_loader), 
                                                                       loss.item()))

Train Epoch:   0 [  0/150 (0%)]	Loss: 1.041497
Train Epoch:   0 [  8/150 (5%)]	Loss: 1.038754
Train Epoch:   0 [ 16/150 (11%)]	Loss: 1.200789
Train Epoch:   0 [ 24/150 (16%)]	Loss: 1.090870
Train Epoch:   0 [ 32/150 (21%)]	Loss: 1.055345
Train Epoch:   0 [ 40/150 (26%)]	Loss: 1.123244
Train Epoch:   0 [ 48/150 (32%)]	Loss: 1.080847
Train Epoch:   0 [ 56/150 (37%)]	Loss: 1.110489
Train Epoch:   0 [ 64/150 (42%)]	Loss: 1.067616
Train Epoch:   0 [ 72/150 (47%)]	Loss: 1.034437
Train Epoch:   0 [ 80/150 (53%)]	Loss: 1.041163
Train Epoch:   0 [ 88/150 (58%)]	Loss: 1.015485
Train Epoch:   0 [ 96/150 (63%)]	Loss: 1.282724
Train Epoch:   0 [104/150 (68%)]	Loss: 1.102610
Train Epoch:   0 [112/150 (74%)]	Loss: 1.001182
Train Epoch:   0 [120/150 (79%)]	Loss: 1.044164
Train Epoch:   0 [128/150 (84%)]	Loss: 1.025715
Train Epoch:   0 [136/150 (89%)]	Loss: 0.989628
Train Epoch:   0 [108/150 (95%)]	Loss: 0.984355
Train Epoch:   1 [  0/150 (0%)]	Loss: 1.044328
Train Epoch:   1 [  8/150 (5%)]	Loss: 0.994

### Save Model

In [0]:
# Save the last model
torch.save(model, f'{MODEL_PATH}.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Testing Process

### Blank Paper

In [0]:
# Lets put us in blank paper condition
del model

### Prediction

In [0]:
logger = logging.getLogger('iris')


## Prediction 
def handle(event, **kwargs):
    # If data is received as json convert to pandas
    event = event['data'] if 'data' in event else event
    if not isinstance(event, pd.DataFrame):
        event = pd.DataFrame.from_dict(event, orient='columns')

    # Convert to NDArray
    data = torch.from_numpy(event.values.astype('float32'))
    
    # Retrieve model from disk and use it for predictions
    model = torch.load(f'{MODEL_PATH}.pt')
    model.eval()
    
    # Target format convertion
    target_dict = {0: 'setosa', 1: 'versicolor', 2:'virginica'}
    to_target = np.vectorize(lambda x: target_dict[x])
    
    return to_target(np.argmax(model(data).detach().numpy(), axis=1)).tolist()

## Testing and liveness check
def test(data, **kwargs):
    pred = handle(data)

    logger.warning(f"predicted: {pred}")
    
    return True


test(iris.data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
predicted: ['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'vers

True